<a href="https://colab.research.google.com/github/NikolaevMikhailRoma/Other-Projects/blob/main/%D0%97%D0%B0%D0%BD%D1%8F%D1%82%D0%B8%D0%B5_9_Room_Geometry_Estimation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import zipfile
import io
import os
from scipy.io import wavfile
import numpy as np
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

from keras.optimizers import Adam
from keras.models import Model
from keras.layers import Input, Dense, Dropout, BatchNormalization, Flatten, Conv1D, Activation
from keras.models import load_model

#Подключаем диски, распаковываем ИХ

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
z = zipfile.ZipFile('/content/drive/MyDrive/UAI/Занятие 9. Room Size Estimation/RIRs2.zip', 'r') #<----- Вот сюда нужно поставить правильный путь к RIRs2.zip
z.extractall()

KeyboardInterrupt: ignored

In [ ]:
!ls RIRs2/

#Функции подготовки xTrain, yTrain

In [4]:
def PrepareTrainData(path2rirs):
  #Инициализируем списки
  xTrain = []
  yTrain = []
  
  for filename in os.listdir(path2rirs): #Перебираем файлы
    roomSize = filename.split('-') #Разбиваем имя файла на строки: [длина, ширина, высота, xx.wav]
    yTrain.append(roomSize[:3]) #Добавляем в yTrain [длина, ширина, высота]
    _, data = wavfile.read(path2rirs + filename) #Читам ИХ
    xTrain.append(data) # Добавляем ИХ в xTrain

  xTrain = np.array(xTrain) #xTrain -> numpy array
  yTrain = np.array(yTrain, dtype=float) #yTrain -> numpy array + конвертируем строки во float

  return xTrain, yTrain

#Полезные функции масштабирования

In [5]:
def getScales(yTrain): #возвращаем [минимум длины, максимум длины, минимум ширины, максимум ширины, минимум высоты, максимум высоты]
  return [min(yTrain[:,0]), max(yTrain[:,0]), min(yTrain[:,1]), max(yTrain[:,1]), min(yTrain[:,2]), max(yTrain[:,2])]

In [6]:
def Transform_yTrain(yTrain, scales): #уTrain -> [0..1]
  yTrain = yTrain.copy()
  yTrain[:,0] = (yTrain[:, 0] - scales[0])/(scales[1] - scales[0])
  yTrain[:,1] = (yTrain[:, 1] - scales[2])/(scales[3] - scales[2])
  yTrain[:,2] = (yTrain[:, 2] - scales[4])/(scales[5] - scales[4])
  return yTrain

In [7]:
def TrasformBack_yTrain(yTrain, scales): #[0..1] -> уTrain
  yTrain = yTrain.copy()
  yTrain[:,0] = yTrain[:,0]*(scales[1] - scales[0]) + scales[0]
  yTrain[:,1] = yTrain[:,1]*(scales[3] - scales[2]) + scales[2]
  yTrain[:,2] = yTrain[:,2]*(scales[5] - scales[4]) + scales[4]
  return yTrain

#Готовим xTrain, yTrain

In [ ]:
xTrain, yTrain = PrepareTrainData('/content/RIRs2/') #читаем файлы, готовим выборки
print(xTrain.shape)
print(yTrain.shape)

In [ ]:
print(yTrain) #размеры комнат

#Scaling

In [ ]:
scales = getScales(yTrain) #читаем по всей выборке макс и мин
print(scales)

In [ ]:
scaled_yTrain = Transform_yTrain(yTrain, scales) #уTrain -> [0..1]
print(scaled_yTrain)

In [ ]:
#Проверяем наши Scaling функции
assert(np.max(scaled_yTrain) <= 1)
assert(np.min(scaled_yTrain) >= 0)
assert(np.max(abs(TrasformBack_yTrain(scaled_yTrain, scales) - yTrain)) < 1e-8)

In [ ]:
xTrain = np.expand_dims(xTrain, axis=2)

In [ ]:
xTrain, xTrainV, yTrain, yTrainV = train_test_split(xTrain, scaled_yTrain, test_size=0.2, shuffle = True) #Перемешиваем и вытаскиваем validation

#Создаем сеть

In [8]:
def CreateModel (xShape, yShapeLength):
  input_impulse = Input(shape=xShape)
  x = Conv1D(10, 4, padding="same", strides=4)(input_impulse)
  x = BatchNormalization()(x)
  x = Activation('relu')(x)

  x = Conv1D(20, 4, padding="same", strides=4)(x)
  x = BatchNormalization()(x)
  x = Activation('relu')(x)

  x = Conv1D(40, 4, padding="same", strides=4)(x)
  x = BatchNormalization()(x)
  x = Activation('relu')(x)

  x = Conv1D(80, 4, padding="same", strides=4)(x)
  x = BatchNormalization()(x)
  x = Activation('relu')(x)

  x = Conv1D(160, 4, padding="same", strides=4)(x)
  x = BatchNormalization()(x)
  x = Activation('relu')(x)

  x = Conv1D(160, 4, padding="same", strides=4)(x)
  x = BatchNormalization()(x)
  x = Activation('relu')(x)

  x = Flatten()(x)
  x = Dropout(0.5)(x)
  x = Dense(40)(x)
  x = Dense(yShapeLength, activation='sigmoid')(x)
  return Model([input_impulse], x)

In [ ]:
model = CreateModel(xShape = (xTrain.shape[1], 1), yShapeLength = yTrain.shape[1])
model.summary()

In [ ]:
model.compile(optimizer=Adam(lr=1e-3), loss='mse', metrics=['accuracy'])
model.fit(xTrain, yTrain, epochs=30, batch_size=150, validation_split = 0.1, verbose = 1)

In [ ]:
model.compile(optimizer=Adam(lr=1e-4), loss='mse', metrics=['accuracy'])
model.fit(xTrain, yTrain, epochs=20, batch_size=150, validation_split = 0.1, verbose = 1)

In [ ]:
modelFileName = '/content/drive/MyDrive/UAI/Занятие 9. Room Size Estimation/RoomSizeEstimationModel.h5'

In [ ]:
model.save(modelFileName)

In [ ]:
model = load_model(modelFileName)

#Проверяем работу модели

In [9]:
def PredictRoomSizes(model, xTrainV, scales):
  yTrainP = model.predict(xTrainV) #Считаем по проверочной выборке
  rooomSizesP = TrasformBack_yTrain(yTrainP, scales) #Возвращаем к метрам
  return rooomSizesP

In [ ]:
roomSizes = PredictRoomSizes(model, xTrainV, scales) #предсказанные размеры

In [ ]:
actualRoomSizes = TrasformBack_yTrain(yTrainV, scales) #реальные размеры

In [ ]:
print(roomSizes)

In [ ]:
delta = roomSizes - actualRoomSizes

In [ ]:
print('Средняя ошибка определения размера помещения (м): ', np.average(delta))

# Lite

In [11]:
z = zipfile.ZipFile('/content/drive/MyDrive/UAI/Занятие 9. Room Size Estimation/RIRs2.zip', 'r') #<----- Вот сюда нужно поставить правильный путь к RIRs2.zip
z.extractall()

In [12]:
z = zipfile.ZipFile('/content/drive/MyDrive/UAI/Занятие 9. Room Size Estimation/RIRs.zip', 'r') #<----- Вот сюда нужно поставить правильный путь к RIRs2.zip
z.extractall()

In [13]:
!ls RIRs2/

Выходные данные были обрезаны до нескольких последних строк (5000).
7.5670-6.4863-5.5741-2.wav   9.5321-7.7061-4.1489-2.wav
7.5670-6.4863-5.5741-3.wav   9.5321-7.7061-4.1489-3.wav
7.5670-6.4863-5.5741-4.wav   9.5321-7.7061-4.1489-4.wav
7.5670-6.4863-5.5741-5.wav   9.5321-7.7061-4.1489-5.wav
7.5670-6.4863-5.5741-6.wav   9.5321-7.7061-4.1489-6.wav
7.5670-6.4863-5.5741-7.wav   9.5321-7.7061-4.1489-7.wav
7.5670-6.4863-5.5741-8.wav   9.5321-7.7061-4.1489-8.wav
7.5670-6.4863-5.5741-9.wav   9.5321-7.7061-4.1489-9.wav
7.5723-6.5690-4.1905-10.wav  9.5340-7.9396-4.8634-10.wav
7.5723-6.5690-4.1905-11.wav  9.5340-7.9396-4.8634-11.wav
7.5723-6.5690-4.1905-12.wav  9.5340-7.9396-4.8634-12.wav
7.5723-6.5690-4.1905-13.wav  9.5340-7.9396-4.8634-13.wav
7.5723-6.5690-4.1905-14.wav  9.5340-7.9396-4.8634-14.wav
7.5723-6.5690-4.1905-15.wav  9.5340-7.9396-4.8634-15.wav
7.5723-6.5690-4.1905-16.wav  9.5340-7.9396-4.8634-16.wav
7.5723-6.5690-4.1905-1.wav   9.5340-7.9396-4.8634-1.wav
7.5723-6.5690-4.1905-2.wav   

##Функции подготовки xTrain, yTrain уменьшаем ИХ

In [15]:
def PrepareTrainData(path2rirs):
  #Инициализируем списки
  xTrain = []
  yTrain = []
  
  for filename in os.listdir(path2rirs): #Перебираем файлы
    roomSize = filename.split('-') #Разбиваем имя файла на строки: [длина, ширина, высота, xx.wav]
    if int(roomSize[3].split('.')[0]) <= 12:
      yTrain.append(roomSize[:3]) #Добавляем в yTrain [длина, ширина, высота]
      _, data = wavfile.read(path2rirs + filename) #Читам ИХ
      xTrain.append(data) # Добавляем ИХ в xTrain

  xTrain = np.array(xTrain) #xTrain -> numpy array
  yTrain = np.array(yTrain, dtype=float) #yTrain -> numpy array + конвертируем строки во float

  return xTrain, yTrain

## Меняем на ступенчатый датасет

In [32]:
# xTrain, yTrain = PrepareTrainData('/content/RIRs2/') #читаем файлы, готовим выборки
xTrain, yTrain = PrepareTrainData('/content/RIRs/') #читаем файлы, готовим выборки

print(xTrain.shape)
print(yTrain.shape)

(36000, 4096)
(36000, 3)


In [33]:
print(yTrain) #размеры комнат

[[6.4211 6.0714 4.2222]
 [9.3684 5.6429 6.    ]
 [8.7368 7.3571 4.2222]
 ...
 [6.2105 7.3571 4.8889]
 [7.8947 6.0714 5.5556]
 [6.8421 5.8571 4.    ]]


##Scaling

In [34]:
scales = getScales(yTrain) #читаем по всей выборке макс и мин
print(scales)

[6.0, 10.0, 5.0, 8.0, 4.0, 6.0]


In [35]:
scaled_yTrain = Transform_yTrain(yTrain, scales) #уTrain -> [0..1]
print(scaled_yTrain)

[[0.105275   0.35713333 0.1111    ]
 [0.8421     0.2143     1.        ]
 [0.6842     0.7857     0.1111    ]
 ...
 [0.052625   0.7857     0.44445   ]
 [0.473675   0.35713333 0.7778    ]
 [0.210525   0.2857     0.        ]]


In [36]:
#Проверяем наши Scaling функции
assert(np.max(scaled_yTrain) <= 1)
assert(np.min(scaled_yTrain) >= 0)
assert(np.max(abs(TrasformBack_yTrain(scaled_yTrain, scales) - yTrain)) < 1e-8)

In [37]:
xTrain = np.expand_dims(xTrain, axis=2)

In [38]:
xTrain, xTrainV, yTrain, yTrainV = train_test_split(xTrain, scaled_yTrain, test_size=0.2, shuffle = True) #Перемешиваем и вытаскиваем validation

## Model compile, test

In [39]:
model = CreateModel(xShape = (xTrain.shape[1], 1), yShapeLength = yTrain.shape[1])
# model.summary()

In [40]:
model.compile(optimizer=Adam(lr=1e-3), loss='mse', metrics=['accuracy'])
model.fit(xTrain, yTrain, epochs=30, batch_size=150, validation_split = 0.1, verbose = 1)

Epoch 1/30
173/173 [==============================] - 4s 15ms/step - loss: 0.0690 - accuracy: 0.5057 - val_loss: 0.0699 - val_accuracy: 0.4587
Epoch 2/30
173/173 [==============================] - 3s 15ms/step - loss: 0.0336 - accuracy: 0.6698 - val_loss: 0.2423 - val_accuracy: 0.3476
Epoch 3/30
173/173 [==============================] - 2s 11ms/step - loss: 0.0235 - accuracy: 0.7312 - val_loss: 0.2433 - val_accuracy: 0.3476
Epoch 4/30
173/173 [==============================] - 2s 11ms/step - loss: 0.0193 - accuracy: 0.7607 - val_loss: 0.1675 - val_accuracy: 0.3542
Epoch 5/30
173/173 [==============================] - 2s 11ms/step - loss: 0.0169 - accuracy: 0.7802 - val_loss: 0.1805 - val_accuracy: 0.3660
Epoch 6/30
173/173 [==============================] - 2s 11ms/step - loss: 0.0154 - accuracy: 0.7882 - val_loss: 0.1125 - val_accuracy: 0.4042
Epoch 7/30
173/173 [==============================] - 2s 10ms/step - loss: 0.0141 - accuracy: 0.7996 - val_loss: 0.0900 - val_accuracy: 0.4635

In [41]:
roomSizes = PredictRoomSizes(model, xTrainV, scales) #предсказанные размеры

225/225 [==============================] - 1s 4ms/step


In [42]:
actualRoomSizes = TrasformBack_yTrain(yTrainV, scales) #реальные размеры

In [43]:
print(roomSizes)

[[8.889414  6.037332  4.506546 ]
 [7.256151  7.174595  5.959443 ]
 [6.515961  7.846062  5.8349714]
 ...
 [6.1770515 7.9219756 5.979327 ]
 [6.1883616 6.2278085 5.9871216]
 [9.272259  7.9094305 4.4165764]]


In [44]:
delta = roomSizes - actualRoomSizes

In [45]:
print('Средняя ошибка определения размера помещения (м): ', np.average(delta))

Средняя ошибка определения размера помещения (м):  -0.052571465940228226
